# Example of Verilog file processing

Configure the path to the Pyverilog directory

In [1]:
%config IPCompleter.greedy=True

In [2]:
import sys
pyverilog_path = "/work/Pyverilog"
sys.path.insert(0, pyverilog_path)

In [76]:
from pyverilog.vparser.parser import VerilogCodeParser
from pyverilog.dataflow.modulevisitor import ModuleVisitor
from pyverilog.dataflow.signalvisitor import SignalVisitor
from pyverilog.dataflow.bindvisitor import BindVisitor
from pyverilog.dataflow.optimizer import VerilogDataflowOptimizer
from pyverilog.dataflow.subset import VerilogSubset

In [4]:
verilog_file_path = "/work/Pyverilog/verilogcode/count.v"
%cat /work/Pyverilog/verilogcode/count.v

module TOP(CLK, RST_X);
  input CLK;
  input RST_X;
  reg [3:0] cnt;

  always @(posedge CLK or negedge RST_X) begin
    if(!RST_X) begin
      cnt <= 0;
    end else begin
      cnt <= cnt + 1;
    end
  end
  
endmodule


In [15]:
parser=VerilogCodeParser([verilog_file_path])
ast = parser.parse()

In [25]:
module_visitor = ModuleVisitor()
module_visitor.visit(ast)
moduleinfotable = module_visitor.get_moduleinfotable()

In [32]:
signal_visitor = SignalVisitor(moduleinfotable, "TOP")
signal_visitor.start_visit()
frametable = signal_visitor.getFrameTable()

In [45]:
bind_visitor = BindVisitor(moduleinfotable, "TOP", frametable, noreorder=False)
bind_visitor.start_visit()
dataflow = bind_visitor.getDataflows()

In [47]:
dataflow.getTerms()

{TOP.CLK: TOP.CLK, TOP.RST_X: TOP.RST_X, TOP.cnt: TOP.cnt}

In [49]:
bd = dataflow.getBinddict()

In [61]:
for bk, bv in sorted(bd.items(), key=lambda x:str(x[0])):
    for bvi in bv:
        print(bvi.tostr())

(Bind dest:TOP.cnt tree:(Branch Cond:(Operator Ulnot Next:(Terminal TOP.RST_X)) True:(IntConst 0) False:(Operator Plus Next:(Terminal TOP.cnt),(IntConst 1))))


In [56]:
bd

{TOP.cnt: [<pyverilog.dataflow.dataflow.Bind at 0x7f61dd222cf8>]}

In [68]:
optimizer = VerilogDataflowOptimizer(dataflow.getTerms(),  dataflow.getBinddict())
optimizer.resolveConstant()

In [74]:
resolved_terms = optimizer.getResolvedTerms()
resolved_binddict = optimizer.getResolvedBinddict()
constlist = optimizer.getConstlist()

In [77]:
subset = VerilogSubset("TOP", dataflow.getTerms(), dataflow.getBinddict(), resolved_terms, resolved_binddict, constlist)

In [102]:
subset.getBindSourceSubset(["TOP.cnt"])

({TOP.cnt: [<pyverilog.dataflow.dataflow.Bind at 0x7f61dd222cf8>]},
 {TOP.RST_X, TOP.cnt})

In [112]:
for left, rights in subset.getBindSourceSubset(["TOP.cnt"])[0].items():
    for right in rights:
        assign_type = subset.getAssignType(left, right)
        print(assign_type)
        print(right)

clockedge


In [106]:
subset.getBindSourceSubset(["TOP.cnt"])[0].items()

dict_items([(TOP.cnt, [<pyverilog.dataflow.dataflow.Bind object at 0x7f61dd222cf8>])])

In [119]:
bind_visitor.visit_Always()

TypeError: visit_Always() missing 1 required positional argument: 'node'